In [ ]:
import altair as alt
from vega_datasets import data
from IPython.display import display

# Load data
iris = data.iris()

# Create chart
chart = alt.Chart(iris).mark_circle().encode(
    x='petalLength',
    y='petalWidth',
    color='species'
)

# Display chart in HTML
chart.save('chart.json')
display(chart)

In [ ]:
from vega_datasets import data
source = data.stocks()
chart = (
    alt.Chart(source)
    .transform_filter(alt.datum.symbol != "IBM")
    .encode(color=alt.Color("symbol", legend=None))
)
line = chart.mark_line().encode(x="date:T", y="price:Q")
label = chart.encode(
    x=alt.X("max(date):T"),
    y=alt.Y("price:Q", aggregate=alt.ArgmaxDef(argmax="date")),
    text="symbol",
)
text = label.mark_text(align="left", dx=4)
circle = label.mark_circle()
chart = line + circle + text
chart.save('chart.json')
display(chart)

In [ ]:
from vega_datasets import data

source = data.stocks()

chart = alt.Chart(source).transform_filter(
    alt.datum.symbol != 'GOOG'
).mark_area().encode(
    x='date:T',
    y='price:Q',
    color='symbol:N',
    row=alt.Row('symbol:N', sort=['MSFT', 'AAPL', 'IBM', 'AMZN'])
).properties(height=50, width=400)
chart.save('chart.json')
display(chart)

In [ ]:
import pandas as pd
import altair as alt

data_path = "../test/covid19data/all_data.csv"
df = pd.read_csv(data_path)
df.drop(index=df[df["province"].isin(["ทั้งประเทศ","ไม่ระบุ"])].index, inplace=True)
obj_cols = df.select_dtypes(include=['object']).columns.to_list()
int_cols = df.select_dtypes(include=['int']).columns.to_list()

Measurement = int_cols[3]
Dimension = obj_cols[1]
Column = obj_cols[2]

chart = alt.Chart(df).mark_bar(clip=True).encode(
    x = alt.X(Dimension,type = "nominal"),
    y = alt.Y(Measurement, 
              type= "quantitative",
              scale= alt.Scale(domain=[df[Measurement].min(),df[Measurement].max()])),
    tooltip = [Dimension,Measurement]
).facet( column = Column
).resolve_scale(x = 'independent',y = 'independent'
)
chart.save('chart.json')
chart

In [ ]:
import altair as alt
# import altair_viewer
import pandas as pd

data_path = "../test/covid19data/all_data.csv"
data_path = "../test/covid19data/total_casesanddeath_by_provinces.csv"
df = pd.read_csv(data_path)

Chart = alt.Chart(df).mark_bar(clip=True).encode(
            x = alt.X("province",type = "nominal", title= "จังหวัด"),
            y = alt.Y("total_case", 
                    type= "quantitative",
                    scale= alt.Scale(domain=[0,df["total_case"].max()]),
                    title= "ผู้ป่วยสะสม"),
            tooltip = ["province","total_case","total_death"]
        ).facet( column = "region"
        ).resolve_scale(x = 'independent',y = 'independent')
        # self.Chart.save('ChartJSON/BarChart.json')
Chart.save('chart.json')
Chart

In [ ]:
import altair as alt
from vega_datasets import data
import json

# Load the unemployment dataset
unemp_data = data.unemployment_across_industries()

# Create an Altair line chart
line_chart = alt.Chart(unemp_data).mark_line().encode(
    x='month:T',
    y='unemployment_rate:Q',
    color='series:N'
)

chart_dict = line_chart.to_dict()

# Convert any Timestamp objects to strings
def datetime_handler(x):
    if isinstance(x, pd.Timestamp):
        return x.strftime('%Y-%m-%dT%H:%M:%S.%fZ')
    raise TypeError('Unknown type')

chart_dict = json.loads(json.dumps(chart_dict, default=datetime_handler))

# Get the data from the original data source
data_dict = unemp_data.to_dict(orient='records')

# Update the chart dictionary to include the data
chart_dict['datasets'] = {'unemployment_data': data_dict}
chart_dict['data'] = {'name': 'unemployment_data'}

# Save the chart dictionary to a JSON file
with open('line_chart.json', 'w') as f:
    json.dump(chart_dict, f)

In [ ]:
import altair as alt
from vega_datasets import data
import json

# with open('Thailandprovinces.json', 'r') as f:
#     Thaiprovince = json.load(f)

data_path = "../test/covid19data/total_cases_by_provinces.csv"
df = pd.read_csv(data_path)
df.drop(index=df[df["province"].isin(["All","None"])].index, inplace=True)
# group = df.groupby('province').transform('sum')
# df['total_case'] = group['total_case']
# df['total_death'] = group['total_death']
# df = df.drop_duplicates(subset=['province'])
# Load the TopoJSON file of Thailand
url = 'https://raw.githubusercontent.com/cvibhagool/thailand-map/master/thailand-provinces.topojson'
topojson_data = alt.topo_feature(url, 'province')
# provincedf = df.loc[:, ['province', 'total_case','total_death','vaccine_total_acm']]
# provincedf['province'] = provincedf['province'].map(Thaiprovince)
chart = alt.Chart(topojson_data).mark_geoshape().encode(
    # color='total_case:Q',
    color = alt.Color('total_case',type='quantitative',
                      scale= alt.Scale(
                        domain = [0,df['total_case'].max()],
                        range = ['white','darkred'],
                        type = 'linear')
                    ),
    tooltip = [alt.Tooltip('properties.NAME_1',type='nominal'),
               alt.Tooltip('total_case',type='quantitative'),
               alt.Tooltip('total_death',type='quantitative')]
).properties(
    width=500,
    height=800
).transform_lookup(
    lookup='properties.NAME_1',
    from_=alt.LookupData(df, 'province', ['total_case','vaccine_total_acm','total_death'])
)
chart.save('chart.json')
# chart

In [ ]:
data_path = "../test/covid19data/total_cases_by_provinces.csv"
df = pd.read_csv(data_path)

l = []
for y in list(set(df['year'])):
    maxweeknum = df.loc[df['year'] == y]['weeknum'].max()
    l.append(df.loc[(df['weeknum'] == maxweeknum) & (df['year'] == y)])
df = pd.concat(l, ignore_index=True)
group = df.groupby('province').transform('sum')
df['total_case'] = group['total_case']
df = df.drop_duplicates(subset=['province'])
# df.loc[df['province'] == 'Nonthaburi']

In [ ]:
import json
data = {
    "Amnat Charoen": "อำนาจเจริญ",
    "Ang Thong": "อ่างทอง",
    "Bueng Kan": "บึงกาฬ",
    "Buri Ram": "บุรีรัมย์",
    "Bangkok Metropolis": "กรุงเทพมหานคร",
    "Chachoengsao": "ฉะเชิงเทรา",
    "Chai Nat": "ชัยนาท",
    "Chaiyaphum": "ชัยภูมิ",
    "Chanthaburi": "จันทบุรี",
    "Chiang Mai": "เชียงใหม่",
    "Chiang Rai": "เชียงราย",
    "Chon Buri": "ชลบุรี",
    "Chumphon": "ชุมพร",
    "Kalasin": "กาฬสินธุ์",
    "Kamphaeng Phet": "กำแพงเพชร",
    "Kanchanaburi": "กาญจนบุรี",
    "Khon Kaen": "ขอนแก่น",
    "Krabi": "กระบี่",
    "Lampang": "ลำปาง",
    "Lamphun": "ลำพูน",
    "Loei": "เลย",
    "Lop Buri": "ลพบุรี",
    "Mae Hong Son": "แม่ฮ่องสอน",
    "Maha Sarakham": "มหาสารคาม",
    "Mukdahan": "มุกดาหาร",
    "Nakhon Nayok": "นครนายก",
    "Nakhon Pathom": "นครปฐม",
    "Nakhon Phanom": "นครพนม",
    "Nakhon Ratchasima": "นครราชสีมา",
    "Nakhon Sawan": "นครสวรรค์",
    "Nakhon Si Thammarat": "นครศรีธรรมราช",
    "Nan": "น่าน",
    "Narathiwat": "นราธิวาส",
    "Nong Bua Lam Phu": "หนองบัวลำภู",
    "Nong Khai": "หนองคาย",
    "Nonthaburi": "นนทบุรี",
    "Pathum Thani": "ปทุมธานี",
    "Pattani": "ปัตตานี",
    "Phangnga": "พังงา",
    "Phatthalung (Songkhla Lake)": "พัทลุง",
    "Phatthalung":"พัทลุง",
    "Phayao": "พะเยา",
    "Phetchabun": "เพชรบูรณ์",
    "Phetchaburi": "เพชรบุรี",
    "Phichit": "พิจิตร",
    "Phitsanulok": "พิษณุโลก",
    "Phrae": "แพร่",
    "Phra Nakhon Si Ayutthaya": "พระนครศรีอยุธยา",
    "Phuket": "ภูเก็ต",
    "Prachin Buri": "ปราจีนบุรี",
    "Prachuap Khiri Khan": "ประจวบคีรีขันธ์",
    "Ranong": "ระนอง",
    "Ratchaburi": "ราชบุรี",
    "Rayong": "ระยอง",
    "Roi Et": "ร้อยเอ็ด",
    "Sa Kaeo": "สระแก้ว",
    "Sakon Nakhon": "สกลนคร",
    "Samut Prakan": "สมุทรปราการ",
    "Samut Sakhon": "สมุทรสาคร",
    "Samut Songkhram": "สมุทรสงคราม",
    "Saraburi": "สระบุรี",
    "Satun": "สตูล",
    "Sing Buri": "สิงห์บุรี",
    "Si Sa Ket": "ศรีสะเกษ",
    "Songkhla": "สงขลา",
    "Songkhla (Songkhla Lake)": "สงขลา",
    "Sukhothai": "สุโขทัย",
    "Suphan Buri": "สุพรรณบุรี",
    "Surat Thani": "สุราษฎร์ธานี",
    "Surin": "สุรินทร์",
    "Tak": "ตาก",
    "Trang": "ตรัง",
    "Trat": "ตราด",
    "Ubon Ratchathani": "อุบลราชธานี",
    "Udon Thani": "อุดรธานี",
    "Uthai Thani": "อุทัยธานี",
    "Uttaradit": "อุตรดิตถ์",
    "Yala": "ยะลา",
    "Yasothon": "ยโสธร"
}

swapped = {v: k for k, v in data.items()}

with open('Thailandprovinces.json', 'w') as f:
    json.dump(swapped, f)

In [ ]:
import altair as alt
import pandas as pd

data_path = "../test/covid19data/total_casesanddeath_by_provinces.csv"
df = pd.read_csv(data_path)
df['date'] = pd.to_datetime(df['year'].astype(str) + '-W' + df['weeknum'].astype(str) + '-0', format='%G-W%V-%w')
df = df.sort_values(by=['date']).loc[df["province"]=="All"].reset_index(drop=True)
# df.loc[df["province"]=="All"].head()
df['allcase'] = 0
df['alldeath'] = 0
for index, row in df.iterrows():
    if index == 0:
        df.loc[index,'allcase'] = row['total_case']
        casenextvalue = df.loc[index,'allcase']
        df.loc[index,'alldeath'] = row['total_death']
        deathnextvalue = df.loc[index,'alldeath']
    elif index == df.shape[0] -1:
        df.loc[index,'allcase'] = casenextvalue + row['new_case']
        df.loc[index,'alldeath'] = deathnextvalue + row['new_death']
    else:
        df.loc[index,'allcase'] = casenextvalue + row['new_case']
        casenextvalue = df.loc[index,'allcase']
        df.loc[index,'alldeath'] = deathnextvalue + row['new_death']
        deathnextvalue = df.loc[index,'alldeath']

In [ ]:
import altair as alt
alt.Chart(df).mark_line(point=alt.OverlayMarkDef(filled=False, fill="white")
                         ).encode(
    x=alt.X("date",type="temporal"),
    y=alt.Y(
        alt.repeat("layer"), aggregate="mean"),
    tooltip = ['date:T','allcase:Q','alldeath:Q'],
    color=alt.datum(alt.repeat("layer")),
).repeat(layer=["allcase", "alldeath"])